In [167]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [168]:
#load data
#data cols are xyz locations of red1, red2, blue1, blue2, yellow1, yellow2
df=pd.read_csv('data.csv')
assert not df.isnull().values.any(), "Null values in df"
assert len(df.columns) == 18, "num cols not 18"

In [169]:
def get_attr(sample):
    sample=sample.to_numpy()

    red1 = (sample[0:3])
    red2 = (sample[3:6])
    blue1 = (sample[6:9])
    blue2 = (sample[9:12])
    yellow1 = (sample[12:15])
    yellow2 = (sample[15:18])

    return red1, red2, blue1, blue2, yellow1, yellow2
    

In [170]:
#using cross product and having box be flat at xy-plane

#input is a tuple of (red1, red2, blue1, blue2, yellow1, yellow2)
def is_inside(input, ratio_w, ratio_d, vis):
    #input=input.to_numpy()
    red1=input[0]
    R1R2=np.subtract(input[1] , input[0]) #line from red1 to red2
    length=np.linalg.norm(R1R2) #distance between red1 and red2
    unit_length = R1R2 / length #unit vector of R1R2
    #print(R1R2)
    
    temp_width=[unit_length[1], -unit_length[0], 0]


    unit_width=temp_width/np.linalg.norm(temp_width)
    unit_depth=np.cross(unit_length, unit_width)

    #floating point error stuff
    assert abs(1 - np.linalg.norm(unit_length)) <= 0.0001, "||unit length|| not 1"
    assert abs(1 - np.linalg.norm(unit_width)) <= 0.0001, "||unit width|| not 1"
    assert abs(1 - np.linalg.norm(unit_depth)) <= 0.0001, "||unit depth|| not 1"

    assert abs(np.dot(unit_width, unit_depth)) <= 0.0001, "dot(w, d) not 0"
    assert (np.dot(unit_length, unit_depth)) <= 0.0001, "dot(l, d) not 0"
    assert (np.dot(unit_width, unit_length)) <= 0.0001, "dot(l, w) not 0"

    RW=unit_width*(length/2)*ratio_w
    RD=unit_depth*(length/2)*ratio_d

    
    
    #always evaluates to true if RW and RD are 0
    def boxchecker(loc):
        if (-np.dot(RW, RW) <= np.dot(RW, np.subtract(loc , red1)) and
            np.dot(RW, np.subtract(loc , red1))  <= np.dot(RW, RW) and

            -np.dot(RD, RD) <= np.dot(RD, np.subtract(loc , red1)) and
            np.dot(RD, np.subtract(loc , red1))  <= np.dot(RD, RD) and

            0<= np.dot(R1R2, np.subtract(loc , red1)) and
            np.dot(R1R2, np.subtract(loc , red1))  <= np.dot(R1R2, R1R2)):

            return True
        return False
    
    
    def visualize():
        red2=input[1]
        vertices=np.array([red1-RW-RD, red1-RW+RD, red1+RW-RD, red1+RW+RD,
                  red2-RW-RD, red2-RW+RD, red2+RW-RD, red2+RW+RD])
        
        faces=[
            [0,1,3,2],
            [0,4,5,1],
            [0,4,6,2],
            [4,5,7,6],
            [2,6,7,3],
            [1,5,7,3]
        ]
       
        ax = plt.axes(projection='3d')
        ax.set_aspect('equal', adjustable='box')
        

        for face in faces:
            x = vertices[face, 0]
            y = vertices[face, 1]
            z = vertices[face, 2]
            poly = Poly3DCollection([list(zip(x, y, z))], alpha=0.2, facecolor='gray', edgecolor='black')
            ax.add_collection3d(poly)
        ax.plot(vertices[:,0], vertices[:,1], vertices[:,2], 'o', color="gray")
        ax.plot(input[0][0], input[0][1], input[0][2], 'o', color="red")
        ax.plot(input[1][0], input[1][1], input[1][2], 'o', color="red")
        ax.plot(input[2][0], input[2][1], input[2][2], 'o', color="blue")
        ax.plot(input[3][0], input[3][1], input[3][2], 'o', color="blue")
        ax.plot(input[4][0], input[4][1], input[4][2], 'o', color="yellow")
        ax.plot(input[5][0], input[5][1], input[5][2], 'o', color="yellow")
        ax.view_init(elev=0, azim=None)
       
        
        plt.show()

    if vis:
        visualize()
       
    
    return boxchecker(input[2]) and boxchecker(input[3]) and boxchecker(input[4]) and boxchecker(input[5])

In [ ]:
#edit these
THRESHOLD = 0.9
WIDTH_STEP=0.1
DEPTH_STEP=0.05
CURR_WIDTH=0
CURR_DEPTH=0


total=len(df)

between_red=0

for index, row in df.iterrows():
    stuff=get_attr(row)
    if is_inside(stuff, 0, 0, False):
        between_red+=1

count=0

while count/between_red < THRESHOLD:
    CURR_WIDTH+=WIDTH_STEP
    CURR_DEPTH+=DEPTH_STEP
    count=0

    for index, row in df.iterrows():
        stuff=get_attr(row)
        if is_inside(stuff, CURR_WIDTH, CURR_DEPTH, False):
            count+=1
    
print(f"total: {total}\namount within red centroids: {between_red}\ncount: {count}\nwidth ratio: {CURR_WIDTH}\ndepth_ratio: {CURR_DEPTH}")






total: 63
amount within red centroids: 49
count: 45
width ratio: 1.3
depth_ratio: 0.65
